# Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные - статистика о платёжеспособности.

##### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

###  №1. Изучим общую информацию о файле:

In [1]:
import pandas as pd

class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Изучим информацию о таблице:

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Взглянув на данные можно сделать вывод о том, что в столбцах days_employed и total_income имеются пропуски, при этом их одинаковое количество. Нмже исследуем возможную причину их появления и степень случайности.

Исследуем столбцы debt/gender/children/education_id/family_status_id/ на наличие артефактов:

In [4]:
def research(value):
    print(color.BOLD + 'Значения столбца', value + ':' + color.END)
    print(df[value].unique())
    print()
research('debt')
research('gender')    
print(color.BOLD + 'Значения столбца children:' + color.END)
print(df['children'].value_counts())    
print()
print(color.BOLD + 'Сгруппированные данные по столбцам education_id и education:' + color.END)
print(df.groupby('education_id')['education'].unique())
print()
print(color.BOLD + 'Сгруппированные данные по столбцам family_status_id и family_status:' + color.END)
print(df.groupby('family_status_id')['family_status'].unique())

Значения столбца debt:
[0 1]

Значения столбца gender:
['F' 'M' 'XNA']

Значения столбца children:
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Сгруппированные данные по столбцам education_id и education:
education_id
0                             [высшее, ВЫСШЕЕ, Высшее]
1                          [среднее, Среднее, СРЕДНЕЕ]
2    [неоконченное высшее, НЕОКОНЧЕННОЕ ВЫСШЕЕ, Нео...
3                    [начальное, НАЧАЛЬНОЕ, Начальное]
4     [Ученая степень, УЧЕНАЯ СТЕПЕНЬ, ученая степень]
Name: education, dtype: object

Сгруппированные данные по столбцам family_status_id и family_status:
family_status_id
0          [женат / замужем]
1         [гражданский брак]
2           [вдовец / вдова]
3                [в разводе]
4    [Не женат / не замужем]
Name: family_status, dtype: object


Из артефактов можно заметить пол XNA в столбце 'gender', отрицательные значения в столбце children, а также аномальное значение в 20 детей в этом же столбце, дубликаты в разном регистре в столбце education, в столбце family_status одно значение написано с заглавной. Продолжим исследовать таблицу.

Проверка значений в столбцах purpose/income_type/education для поиска потенциальных дубликатов:

In [5]:
def values_unique(col):
    print(color.BOLD + 'Значения столбца', col +':' + color.END)
    print(' | '.join(df[col].unique()))
    print()

values_unique('purpose')
values_unique('income_type')
values_unique('education')
values_unique('family_status')

Значения столбца purpose:
покупка жилья | приобретение автомобиля | дополнительное образование | сыграть свадьбу | операции с жильем | образование | на проведение свадьбы | покупка жилья для семьи | покупка недвижимости | покупка коммерческой недвижимости | покупка жилой недвижимости | строительство собственной недвижимости | недвижимость | строительство недвижимости | на покупку подержанного автомобиля | на покупку своего автомобиля | операции с коммерческой недвижимостью | строительство жилой недвижимости | жилье | операции со своей недвижимостью | автомобили | заняться образованием | сделка с подержанным автомобилем | получение образования | автомобиль | свадьба | получение дополнительного образования | покупка своего жилья | операции с недвижимостью | получение высшего образования | свой автомобиль | сделка с автомобилем | профильное образование | высшее образование | покупка жилья для сдачи | на покупку автомобиля | ремонт жилью | заняться высшим образованием

Значения столбца inc

Сразу бросается в глаза столбец purpose, цели займа не сведены к категориям, также видно количество дубликатов в столбце education и одно значение family_status с заглавной буквы.

Проверка наличия и количества отрицательных значений в других столбцах:

In [6]:
def negative_value(val):
    print('Количество отрицательных значений в столбце', val + ':', '{}'.format(df[df[val] < 0][val].count()))
negative_value('days_employed')
negative_value('dob_years')
negative_value('total_income')

Количество отрицательных значений в столбце days_employed: 15906
Количество отрицательных значений в столбце dob_years: 0
Количество отрицательных значений в столбце total_income: 0


Изучим записи по столбцу children со значением '20'

In [7]:
df20 = df[df['children'] == 20]
df20.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля


Вероятно из-за человеческого фактора были введены неверные значения, возможно подразумевалось число 2, так как данные не обладают общими свойствами, удалим аномальные значения из таблицы.

In [8]:
df = df[df['children'] != 20]
df.info()
print()
print(df['children'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 12 columns):
children            21449 non-null int64
days_employed       19284 non-null float64
dob_years           21449 non-null int64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        19284 non-null float64
purpose             21449 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB

 0    14149
 1     4818
 2     2055
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64


Пропущенные значения успешно удалены.

In [9]:
df.isnull().sum()

children               0
days_employed       2165
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2165
purpose                0
dtype: int64

Изучим значение XNA столбца gender:

In [10]:
dfxna = df[df['gender'] == 'XNA']
dfxna

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Вероятно это значение также попало по ошибке, так как человек, заполнявший форму забыл указать пол. Можно поставить любой пол, либо удалить запись, в обоих случаях это не повлияет на выборку. Тем не менее, чтобы минимизировать внешнее вмешательство, удалим эту запись.

In [11]:
df = df[df['gender'] != 'XNA']

### Вывод

#### В файле можно заметить следующие артефакты: 

1. отрицательные значения в столбцах days_employed и children;

2. наличе дубликатов в разном регистре в столбце education; 

3. Пропуски в полях days_employed и total_income;

4. В столбце purpose находятся похожие значения, не сведенные к категории;

5. В столбце family_status все значения, кроме последнего, в нижнем регистре;

6. Cтолбцы days_employed и total_income преобразовались в тип float, из-за того, что туда попали значения NaN.

7. Значение 'XNA' в столбце gender, которое попало по ошибке. Запись была удалена.

8. 76 записей со значением '20' в столбце children. Значение попало в таблицу случайно, так как выборка не обладает общими свойствами: например, встречаются заёмщики с возрастом в 21/27 лет. Данные были удалены.

9. Пропуски находятся в строках с одинаковым индексом столбцов total_income и days_employed, вероятно значения также попали из-за технического сбоя.

### №2. Предобработка данных

Изучим общее количество уникальных и пропущенных значений по столбцу income_type:

In [12]:
print(color.BOLD + 'Количество уникальных значений в столбце income_type:' + color.END)
print(df['income_type'].value_counts())
print()
print(color.BOLD + 'Общая сумма уникальных значений в столбце income_type: {}'.format(df['income_type'].value_counts().sum()) + color.END)
print()
print(color.BOLD + 'Сгруппированные значения total_income по столбцу income_type:' + color.END)
print(df.groupby('income_type')['total_income'].count().sort_values(ascending=False))
print()
print(color.BOLD + 'Общая сумма сгруппированных значений total_income в столбце income_type: {}'.format(df.groupby('income_type')['total_income'].count().sum()) + color.END)
print()

print(color.BOLD + 'Сгруппированные значения days_employed по столбцу income_type:' + color.END)
print(df.groupby('income_type')['days_employed'].count().sort_values(ascending=False))
print()
print(color.BOLD + 'Общая сумма сгруппированных значений days_employed в столбце income_type: {}'.format(df.groupby('income_type')['days_employed'].count().sum()) + color.END)
print()

Количество уникальных значений в столбце income_type:
сотрудник          11076
компаньон           5062
пенсионер           3847
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Общая сумма уникальных значений в столбце income_type: 21448

Сгруппированные значения total_income по столбцу income_type:
income_type
сотрудник          9974
компаньон          4557
пенсионер          3436
госслужащий        1311
безработный           2
студент               1
предприниматель       1
в декрете             1
Name: total_income, dtype: int64

Общая сумма сгруппированных значений total_income в столбце income_type: 19283

Сгруппированные значения days_employed по столбцу income_type:
income_type
сотрудник          9974
компаньон          4557
пенсионер          3436
госслужащий        1311
безработный           2
студент               1
предприниматель       1
в декрете             1
Name

Становится очевидно, что значения отсутствуют в строках с одинаковыми индексами столбцов days_employed и total_income.
Изучим, по каким значениям income_type встречаются NaN

In [13]:
print(color.BOLD +'Группировка пропущенных значений по столбцу income_type:' + color.END)
dfincomenan = df['income_type'].value_counts() - df.groupby('income_type')['total_income'].count()
#По столбцу days_employed значения совпадают.
#dfdaysnan = df['income_type'].value_counts() - df.groupby('income_type')['days_employed'].count()
print(dfincomenan.sort_values(ascending = False))
#print(dfdaysnan.sort_values(ascending = False))
print()
print(color.BOLD + 'Сумма всех пропущенных значений: {}'.format(dfincomenan.sum()) + color.END)
print()

Группировка пропущенных значений по столбцу income_type:
сотрудник          1102
компаньон           505
пенсионер           411
госслужащий         146
предприниматель       1
студент               0
в декрете             0
безработный           0
dtype: int64

Сумма всех пропущенных значений: 2165



Пропуски можно считать абсолютно случайными, иначе мы бы увидели закономерность в пропущенных данных, а именно логично было бы указать days_employed NaN или 0, например, только для пенсионеров/студентов. Скорее всего пропуски появились в результате технического сбоя. Сами пропущенные значения подсчитаны верно, поскольку сумма совпадает со значениями в df.insull().sum().

Изучим средние и медианные значения, которыми будем заполнять пропущенные данные:

In [14]:
income_all = df['total_income'].agg({'mean', 'median', 'min', 'max'}).astype(int)
print(color.BOLD +'Полученные значения по всему столбцу total_income:' + color.END)
income_all

Полученные значения по всему столбцу total_income:


median     145017
max       2265604
mean       167415
min         20667
Name: total_income, dtype: int64

In [15]:
income_bytype = df.groupby('income_type')['total_income'].agg({'mean', 'median', 'min', 'max'}).astype(int)
print(color.BOLD + 'Величины, распределенные по столбцу income_type:' + color.END)
income_bytype

Величины, распределенные по столбцу income_type:


,median,max,mean,min
income_type,,,,
безработный,131339,202722,131339,59956
в декрете,53829,53829,53829,53829
госслужащий,150420,910451,170831,29200
компаньон,172434,2265604,202522,28702
пенсионер,118406,735103,137150,20667
предприниматель,499163,499163,499163,499163
сотрудник,142594,1726276,161345,21367
студент,98201,98201,98201,98201


Таким образом, заполнять пропуски медианными/средними значениями со всего столбца будет неверно, поскольку они либо выше, либо ниже медианных/средних ззначений по конкретной категории income_type, соотвественно стандартное отклонение будет больше, а это увеличит погрешность. Для заполнения пропусков будем использовать медиану, поскольку среднее может смещаться ближе к max или min из-за нескольких очень больших или очень маленьких значений.

Заполним пропущенные значения в столбце total_income:

In [16]:
list_oftype = ['пенсионер', 'сотрудник', 'предприниматель', 'компаньон', 'госслужащий']
def fill_nan(column_type, value):
    median = df[df['income_type'] == value][column_type].median()
    df.loc[(df[column_type].isna()) & (df['income_type'] == value), column_type] = median
    print('Заполнили пропущенные значения категории', "'" + value + "'", 'по столбцу', column_type, 'остальось заполнить: {} пропуска'.format(pd.isnull(df[column_type]).sum()))
     
for i in list_oftype:
    fill_nan('total_income', i)

Заполнили пропущенные значения категории 'пенсионер' по столбцу total_income остальось заполнить: 1754 пропуска
Заполнили пропущенные значения категории 'сотрудник' по столбцу total_income остальось заполнить: 652 пропуска
Заполнили пропущенные значения категории 'предприниматель' по столбцу total_income остальось заполнить: 651 пропуска
Заполнили пропущенные значения категории 'компаньон' по столбцу total_income остальось заполнить: 146 пропуска
Заполнили пропущенные значения категории 'госслужащий' по столбцу total_income остальось заполнить: 0 пропуска


Проделаем то же самое для столбца days_employed:
сперва приведем отрицательные значения к положительным, проделаем такую же операцию для столбца 'children'.

In [17]:
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()
#проверим остались ли отрицательные значения
negative_value('days_employed')
negative_value('children')
#Убедимся, что данные преобразованы
df.head()

Количество отрицательных значений в столбце days_employed: 0
Количество отрицательных значений в столбце children: 0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Рассчитаем средние/медианные/максимальные и минимальные значения для столбца days_employed по столбцу income_type:

In [18]:
income_bydays = df.groupby('income_type')['days_employed'].agg({'mean', 'median', 'min', 'max'}).astype(int)
print(color.BOLD + 'Величины, распределенные по столбцу days_employed:' + color.END)
income_bydays

Величины, распределенные по столбцу days_employed:


,median,max,mean,min
income_type,,,,
безработный,366413,395302,366413,337524
в декрете,3296,3296,3296,3296
госслужащий,2689,15193,3398,39
компаньон,1554,17615,2116,30
пенсионер,365249,401755,365011,328728
предприниматель,520,520,520,520
сотрудник,1573,18388,2325,24
студент,578,578,578,578


По столбцу days_employed видно, что такие строки как 'пенсионер' и 'безработный' принимают аномальные значения, заменим их на '0' (подразумевая, что они не работают согласно законодательству, поэтому значение равно 0), так же здесь нелогичное значение для 'в декрете', заменим его тоже на 0. Остальные пропущенные значения заменим на медианные.

In [19]:
list_oftype2 =  ['пенсионер', 'безработный', 'в декрете']
list_oftype3 = ['сотрудник', 'компаньон', 'госслужащий', 'предприниматель']
#Замена аномальных значений на 0
for i in list_oftype2:
    df.loc[(df['income_type'] == i), 'days_employed'] = 0
print('Заменили все значения для "пенсионер", "в декрете", "безработный" на "0", остался {} пропуск(а).'.format(pd.isnull(df['days_employed']).sum()))
print()
#Заполним пропуски в оставшихся категориях медианными значениями
for i in list_oftype3:
    fill_nan('days_employed', i)
    
#проверим, остались ли пропущенные значения в датафрейме
df.isnull().sum()

Заменили все значения для "пенсионер", "в декрете", "безработный" на "0", остался 1754 пропуск(а).

Заполнили пропущенные значения категории 'сотрудник' по столбцу days_employed остальось заполнить: 652 пропуска
Заполнили пропущенные значения категории 'компаньон' по столбцу days_employed остальось заполнить: 147 пропуска
Заполнили пропущенные значения категории 'госслужащий' по столбцу days_employed остальось заполнить: 1 пропуска
Заполнили пропущенные значения категории 'предприниматель' по столбцу days_employed остальось заполнить: 0 пропуска


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

1. Были обнаружены пропущенные значения в столбцах total_income и days_employed в строках с одинаковыми индексами. Поскольку, какая-либо зависимость между пропущенными данными и значениями других столбцов отсуствовала, можно сделать вывод, что это абсолютно случайные пропуски, вызыванные техническими проблемами.
2. Пропуски из столбцов total_income и days_employed заполнены медианными значениями по категориям income_type, так как медиана по всему столбцу увеличила бы стандартное отклонение и погрешность. Аномальные значения в столбце days_employed по категориям 'пенсионер', 'безработный', 'в декрете', заменены на 0. Вероятно они также попали туда в результате технического сбоя.
3. Отрицательные значения столбцов days_employed и children были заменены на положительные. 

### Замена типа данных:

In [20]:
#Изучим типы данных в таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
children            21448 non-null int64
days_employed       21448 non-null float64
dob_years           21448 non-null int64
education           21448 non-null object
education_id        21448 non-null int64
family_status       21448 non-null object
family_status_id    21448 non-null int64
gender              21448 non-null object
income_type         21448 non-null object
debt                21448 non-null int64
total_income        21448 non-null float64
purpose             21448 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.8+ MB


Данные соотвествуют типам по всем столбцам кроме days_employed и income_type. Заменим их на целочисленный тип:

In [21]:
df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce')
df['days_employed'] = pd.to_numeric(df['days_employed'], errors='coerce')
df['total_income'] = df['total_income'].astype(int)
df['days_employed'] = df['days_employed'].astype(int)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
children            21448 non-null int64
days_employed       21448 non-null int64
dob_years           21448 non-null int64
education           21448 non-null object
education_id        21448 non-null int64
family_status       21448 non-null object
family_status_id    21448 non-null int64
gender              21448 non-null object
income_type         21448 non-null object
debt                21448 non-null int64
total_income        21448 non-null int64
purpose             21448 non-null object
dtypes: int64(7), object(5)
memory usage: 2.8+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Тип float был заменен на int в столбцах days_employed и income_type. Для замены были выбраны методы to_numeric() и .astype().  При наличии строковых данных to_numeric() преобразовал бы в вещественный тип, метод astype() преобразовал все в целочисленный. Аргумент errors='coerce' мог привести к появлению к пропускам в данных, но по количеству значений видно, что пропуски не образовались. Теперь данные в этих столбцах стали более читаемыми.

### Обработка дубликатов: 
Еще раз посмотрим уникальные значения столбца 'education' 

In [22]:
values_unique('education')

Значения столбца education:
высшее | среднее | Среднее | СРЕДНЕЕ | ВЫСШЕЕ | неоконченное высшее | начальное | Высшее | НЕОКОНЧЕННОЕ ВЫСШЕЕ | Неоконченное высшее | НАЧАЛЬНОЕ | Начальное | Ученая степень | УЧЕНАЯ СТЕПЕНЬ | ученая степень



Дублированные значения столбца находятся в разном регистре, но имеют одинаковую начальную форму. Таким образом, чтобы убрать все дубликаты в этом столбце, нам достаточно привести все значения к нижнему регистру. Также обрабоботаем столбец 'family_status'.

In [23]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
#Проверка
print(df['education'].value_counts())
print()
values_unique('family_status')

среднее                15173
высшее                  5246
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Значения столбца family_status:
женат / замужем | гражданский брак | вдовец / вдова | в разводе | не женат / не замужем



Проверим наличие дублированных значений:

In [24]:
print(color.BOLD + 'Количество дублированных значений: {}'.format(df.duplicated().sum()) + color.END)
#df[df.duplicated(keep=False)].count()
df[df.duplicated(keep=False)].head(50)

Количество дублированных значений: 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,1573,46,среднее,1,женат / замужем,0,F,сотрудник,0,142594,высшее образование
520,0,1573,35,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
541,0,1573,57,среднее,1,женат / замужем,0,F,сотрудник,0,142594,сделка с подержанным автомобилем
554,0,1573,60,среднее,1,женат / замужем,0,M,сотрудник,0,142594,покупка недвижимости
680,1,2689,30,высшее,0,женат / замужем,0,F,госслужащий,0,150420,покупка жилья для семьи
1005,0,0,62,среднее,1,женат / замужем,0,F,пенсионер,0,118406,ремонт жилью
1191,0,0,61,среднее,1,женат / замужем,0,F,пенсионер,0,118406,операции с недвижимостью
1431,0,1573,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
1511,0,0,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,118406,дополнительное образование
1681,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,118406,на проведение свадьбы


In [25]:
#Удалим дубликаты
df = df.drop_duplicates().reset_index(drop = True)

print('Количество дублированных значений: {}'.format(df.duplicated().sum()))
print(df.info())


Количество дублированных значений: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21377 entries, 0 to 21376
Data columns (total 12 columns):
children            21377 non-null int64
days_employed       21377 non-null int64
dob_years           21377 non-null int64
education           21377 non-null object
education_id        21377 non-null int64
family_status       21377 non-null object
family_status_id    21377 non-null int64
gender              21377 non-null object
income_type         21377 non-null object
debt                21377 non-null int64
total_income        21377 non-null int64
purpose             21377 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

Таким образом методом .str.lower() были приведены значения столбца 'education' к нижнему регистру, таким образом были убраны все дублированные значения столбца. Возможная причина их появления - человеческий фактор, запись вручную в разном регистре. При помощи метода .duplicated() проверил и изучил значения, дубликаты из датафрейма удалил методом df.drop_duplicates().reset_index(drop = True)

### Лемматизация
Проведем леммтизацию столбца purpose, сохраним изученные значения в столбец lemmatized_purpose, попробуем выделить категории из контекста обработанного столбца, изучив содержание. В качестве первого шага лемматизируем уникальные значения purpose, посчитаем и изучим их.

In [26]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [27]:
purpose_rows = ' '.join((df['purpose']).unique())
lemmas = m.lemmatize(purpose_rows)
lemmas = Counter(lemmas)
lemmas

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Можно выделить наиболее часто встречающиеся существительные:
- 'покупка': 10
- 'недвижимость': 10,
- 'образование': 9
- 'жилье': 7

Также интересные для анализа редкие существительные:
- 'строительство': 3
- 'ремонт': 1
- 'свадьба': 3
- 'операция': 4

Поскольку без понимания того, в каком контексте встречаются слова, сложно выделить категории, лемматизируем столбец 'purpose' и изучим его содержание относительно выбранных существительных.

In [28]:
def lemm(purpose_rows):
    return m.lemmatize(purpose_rows)

df['lemmatized_purpose'] = df['purpose'].apply(lemm)

print(df['lemmatized_purpose' ].value_counts())

[автомобиль, \n]                                          970
[свадьба, \n]                                             790
[на,  , проведение,  , свадьба, \n]                       764
[сыграть,  , свадьба, \n]                                 761
[операция,  , с,  , недвижимость, \n]                     674
[покупка,  , коммерческий,  , недвижимость, \n]           658
[покупка,  , жилье,  , для,  , сдача, \n]                 650
[операция,  , с,  , жилье, \n]                            648
[операция,  , с,  , коммерческий,  , недвижимость, \n]    646
[покупка,  , жилье, \n]                                   643
[жилье, \n]                                               642
[покупка,  , жилье,  , для,  , семья, \n]                 637
[недвижимость, \n]                                        632
[строительство,  , собственный,  , недвижимость, \n]      629
[операция,  , со,  , свой,  , недвижимость, \n]           626
[строительство,  , жилой,  , недвижимость, \n]            623
[покупка

По столбцам видно, что жилье и недвижимость по смыслу одни и теже слова, покупка и строительство также подразумевают один и тот же смысл, поэтому можем исключить их и сделать акцент на слове 'недвижимость'. Поскольку 'операция' не медицинская, также исключаем её из списка. Включим в отдельную категорию ремонт. Также оставим автомобиль, свадьбу и образование.

Исходя из изученных данных, можно выделить 5 категорий:
1. свадьба
2. автомобиль
3. недвижимость
4. образование
5. ремонт

Заменим эти лемматизированные значения столбца lemmatized_purpose на новые категории:   

In [29]:
list_purpose = ['свадьба', 'автомобиль', 'недвижимость', 'ремонт',  'образование', 'жилье']

def row_changing(row):
    for i in list_purpose:
        if i in row:
            if i=='жилье':
                if 'ремонт' in row:
                    return 'ремонт'
                return 'недвижимость'
            return i
        
#Такая же функция со схожим принципом работы, но другой реализации, для кроссчека результатов заполнения.
#def row_changing(row):
#    if 'свадьба' in row:
#        return 'свадьба'
#    if 'автомобиль' in row:
#        return 'покупка автомобиля'
#    if 'недвижимость' in row or 'жилье' in row and 'ремонт' not in row:
#        return 'покупка/строительство недвижимости'
#    if 'образование' in row:
#        return 'образовательные цели'
#    if 'ремонт' in row:
#        return 'ремонт'
df['lemmatized_purpose'] = df['lemmatized_purpose'].apply(row_changing)
print(df['lemmatized_purpose'].value_counts())
print()
#Проверим, что мы ничего не упустили и все значения учтены:
print(color.BOLD + 'Сумма всех измененных значений столбца (должна соотвествовать числу строк df): {}'.format(df['lemmatized_purpose'].value_counts().sum()) + color.END)
print()
print(color.BOLD + 'Количество образовавшихся пропусков (при условии, что ключевое слово не попало в значения lemmatized_purpose): {}'.format(df['lemmatized_purpose'].isna().sum()) + color.END)


недвижимость    10169
автомобиль       4290
образование      3998
свадьба          2315
ремонт            605
Name: lemmatized_purpose, dtype: int64

Сумма всех измененных значений столбца (должна соотвествовать числу строк df): 21377

Количество образовавшихся пропусков (при условии, что ключевое слово не попало в значения lemmatized_purpose): 0


Убедимся, что полученные категории были верно распределены:

In [30]:
df.groupby('purpose')['lemmatized_purpose'].unique()

purpose
автомобили                                  [автомобиль]
автомобиль                                  [автомобиль]
высшее образование                         [образование]
дополнительное образование                 [образование]
жилье                                     [недвижимость]
заняться высшим образованием               [образование]
заняться образованием                      [образование]
на покупку автомобиля                       [автомобиль]
на покупку подержанного автомобиля          [автомобиль]
на покупку своего автомобиля                [автомобиль]
на проведение свадьбы                          [свадьба]
недвижимость                              [недвижимость]
образование                                [образование]
операции с жильем                         [недвижимость]
операции с коммерческой недвижимостью     [недвижимость]
операции с недвижимостью                  [недвижимость]
операции со своей недвижимостью           [недвижимость]
покупка жилой недвижимо

### Вывод

Была проведена лемматизация целей получения кредита при помощи библиотеки pymystem3. Полученные леммы были изучены, из них были выделены основные категории и записаны в отдельный столбец 'lemmatized_purpose'.

### Категоризация данных
Для категоризации данных по столбцу total_income найдем по нему квартили:

In [31]:
print(df['total_income'].describe().astype(int))

q4 = df['total_income'].max() 
q3 = df['total_income'].quantile(.75).astype(int)
q2 = df['total_income'].quantile(.50).astype(int)
q1 = df['total_income'].quantile(.25).astype(int)

count      21377
mean      165314
std        98260
min        20667
25%       107523
50%       142594
75%       195785
max      2265604
Name: total_income, dtype: int64


Таким образом Q1 = 107760, Q2 = 142594, Q3 = 195528, Q4 = 2265604. Создадим функцию и разобьем квартили по категориям: 'низкий доход', 'средний доход', 'высокий доход' и 'очень высокий доход'.


In [32]:
quantile_list = [q1, q2, q3, q4]

def categorize_income(income):
    for i in quantile_list:
        if income <= i:
            if i == q1:
                return 'низкий доход'
            if i == q2:
                return 'средний доход'
            if i == q3:
                return 'высокий доход'
            else:
                return 'очень высокий доход'
            
df['income_category'] = df['total_income'].apply(categorize_income)
#Убедимся, что значения правильно распределились по категориям:
print(color.BOLD + 'Ключевые значения total_icnome для каждой категории:' + color.END)
print(df.groupby('income_category')['total_income'].agg({'mean', 'min', 'max'}).astype(int))

Ключевые значения total_icnome для каждой категории:
                         max    mean     min
income_category                             
высокий доход         195785  166763  142595
низкий доход          107523   80916   20667
очень высокий доход  2265604  286608  195799
средний доход         142594  127835  107546


Выделим категории 'есть дети'/'нет детей' по столбцу 'children'. Затем распределим заемщиков по наличию детей и задолженности:

In [33]:
def categorize_children(number_of_children):
    if number_of_children > 0:
        return 'есть дети'
    else:
        return 'нет детей'

df['children_category'] = df['children'].apply(categorize_children)  
#print(df['children_category'].value_counts())

#Функция для распределения по категории наличия задолженности и наличия/отсуствия детей:
children_list = df['children_category'].unique()
df_dict_chd = df.loc[:, ('children_category', 'debt')]

def category_bychild(row):
    children = row['children_category']
    debt = row['debt']
    for i in children_list:
        if i in children:
            if debt == 0:
                return 'нет задолженности'
            else:
                return 'задолженность'
            
df_dict_chd['chdild_debt'] = df_dict_chd.apply(category_bychild, axis=1)
print(color.BOLD + 'Распределение заёмщиков по наличию детей и задолженности:' + color.END)
print(df_dict_chd.groupby('children_category')['chdild_debt'].value_counts()) 

Распределение заёмщиков по наличию детей и задолженности:
children_category  chdild_debt      
есть дети          нет задолженности     6617
                   задолженность          670
нет детей          нет задолженности    13027
                   задолженность         1063
Name: chdild_debt, dtype: int64


Создадим функцию для распределения заёмщиков по уровню дохода и задолженности:

In [34]:
#Функция для распределения по категории наличия задолженности и уровнем дохода:
list_income = df['income_category'].unique()
df_dict_ind = df.loc[:, ('income_category', 'debt')]
def category_byincome(row):
    income = row['income_category']
    debt = row['debt']
    for i in list_income:
        if i in income:
            if debt == 0:
                return 'нет задолженности'
            else:
                return 'задолженность'

df_dict_ind['income_debt'] = df_dict_ind.apply(category_byincome, axis=1)
print(color.BOLD + 'Распределение заёмщиков по уровню дохода и наличию задолженности:' + color.END)
print(df_dict_ind.groupby('income_category')['income_debt'].value_counts())

Распределение заёмщиков по уровню дохода и наличию задолженности:
income_category      income_debt      
высокий доход        нет задолженности    4782
                     задолженность         445
низкий доход         нет задолженности    4918
                     задолженность         427
очень высокий доход  нет задолженности    4963
                     задолженность         381
средний доход        нет задолженности    4981
                     задолженность         480
Name: income_debt, dtype: int64


Создадим функцию для распределения заёмщиков по семейному статусу и задолженности:

In [35]:
list_familyst = df['family_status'].unique()
df_dict_fmd = df.loc[:, ('family_status', 'debt')]

def category_byfamily(row):
    family_status = row['family_status']
    debt = row['debt']
    for i in list_familyst:
        if i in family_status:
            if debt == 0:
                return 'нет задолженности'
            else:
                return 'задолженность'    
    
df_dict_fmd['family_debt'] = df_dict_fmd.apply(category_byfamily, axis=1)
print(color.BOLD + 'Распределение заёмщиков по семейному положению и наличию задолженности:' + color.END)
print(df_dict_fmd.groupby('family_status')['family_debt'].value_counts())

Распределение заёмщиков по семейному положению и наличию задолженности:
family_status          family_debt      
в разводе              нет задолженности     1109
                       задолженность           84
вдовец / вдова         нет задолженности      892
                       задолженность           63
гражданский брак       нет задолженности     3753
                       задолженность          385
женат / замужем        нет задолженности    11362
                       задолженность          928
не женат / не замужем  нет задолженности     2528
                       задолженность          273
Name: family_debt, dtype: int64


Создадим функцию для распределения заёмщиков по целям кредита и задолженности:

In [36]:
list_purpose = df['lemmatized_purpose'].unique()
df_dict_purpose = df.loc[:, ('lemmatized_purpose', 'debt')]

def category_bypurpose(row):
    dict_purp = row['lemmatized_purpose']
    debt = row['debt']
    for i in list_purpose:
        if i in dict_purp:
            if debt == 0:
                return 'нет задолженности'
            else:
                return 'задолженность'    
            
df_dict_purpose['purpose_debt'] = df_dict_purpose.apply(category_bypurpose, axis=1)
df['debt_group'] = df_dict_purpose['purpose_debt']
print(color.BOLD + 'Распределение заёмщиков по целям кредита и наличию задолженности:' + color.END)
print(df_dict_purpose.groupby('lemmatized_purpose')['purpose_debt'].value_counts())   

Распределение заёмщиков по целям кредита и наличию задолженности:
lemmatized_purpose  purpose_debt     
автомобиль          нет задолженности    3889
                    задолженность         401
недвижимость        нет задолженности    9424
                    задолженность         745
образование         нет задолженности    3629
                    задолженность         369
ремонт              нет задолженности     570
                    задолженность          35
свадьба             нет задолженности    2132
                    задолженность         183
Name: purpose_debt, dtype: int64


In [37]:
#Проверим наличие и значения добавленных категорий в основной таблице
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose,income_category,children_category,debt_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,очень высокий доход,есть дети,нет задолженности
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход,есть дети,нет задолженности
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий доход,нет детей,нет задолженности
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,очень высокий доход,есть дети,нет задолженности
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий доход,нет детей,нет задолженности


### Вывод

Были выделены категории по уровню дохода по квартилям:
1. 'низкий доход', соответствующий значениям с 0го по 25й процентиль столбца 'total_income';
2. 'средний доход', соответствующий значению 25го по 50й процентиль столбца 'total_income';
3. 'высокий доход', соответствующий значению с 50го по 75й процентиль столбца 'total_income';
4. 'очень высокий доход', соответствующий значению c 75го по 100й процентиль столбца 'total_income';

Также были выделены категории по наличию детей, такие как: 'есть дети' и 'нет детей'. Были распределены столбцы children_category/family_status/income_category/lemmatized_purpose по наличию задолженности. Для этого брались параметры debt (0 или 1) и соотвествующее этому параметру значения столбцов, таким образом получилось выделить новые значения по наличию задолженности в отдельные столбцы.

### №3. Ответим на вопросы:

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
print(color.BOLD + 'Доля задолженностей сгруппированных по составу семьи:' + color.END)
data_pivot_children =  df.pivot_table(index=['children_category'], columns='debt_group', values='total_income', aggfunc='count')
data_pivot_children['ratio'] = data_pivot_children['задолженность'] / (data_pivot_children['нет задолженности'] + data_pivot_children['задолженность'])
print(data_pivot_children.sort_values(by='ratio', ascending = False))

Доля задолженностей сгруппированных по составу семьи:
debt_group         задолженность  нет задолженности     ratio
children_category                                            
есть дети                    670               6617  0.091945
нет детей                   1063              13027  0.075444


### Вывод

Из полученных данных можно сделать вывод, что семьи, в которых есть дети, на 1.65% реже возвращают кредит в срок, чем семьи, в которых нет детей. 
Вероятность просрочки возврата кредита:
- для семьи с детьми составляет: 9.19%
- для семьи без детей составляет: 7.54% 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
print(color.BOLD + 'Доля задолженностей сгруппированных по семейному положению:' + color.END)
data_pivot_family =  df.pivot_table(index=['family_status'], columns='debt_group', values='total_income', aggfunc='count')
data_pivot_family['ratio'] = data_pivot_family['задолженность'] / (data_pivot_family['нет задолженности'] + data_pivot_family['задолженность'])
print(data_pivot_family.sort_values(by='ratio', ascending = False))

Доля задолженностей сгруппированных по семейному положению:
debt_group             задолженность  нет задолженности     ratio
family_status                                                    
не женат / не замужем            273               2528  0.097465
гражданский брак                 385               3753  0.093040
женат / замужем                  928              11362  0.075509
в разводе                         84               1109  0.070411
вдовец / вдова                    63                892  0.065969


### Вывод

Согласно полученной информации, чаще всего возвращают кредит в срок вдовцы/вдовы, разведенные и женатые/замужние заёмщики. 
Реже всех возвращают в срок не женатые, для них вероятность просрочки составляет 9.74% и лица, состоящие в гражданском браке, для них вероятность просрочки 9.30%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
print(color.BOLD + 'Доля задолженностей сгруппированных по уровню дохода:' + color.END)
data_pivot_income =  df.pivot_table(index=['income_category'], columns='debt_group', values='total_income', aggfunc='count')
data_pivot_income['ratio'] = data_pivot_income['задолженность'] / (data_pivot_income['нет задолженности'] + data_pivot_income['задолженность'])
print(data_pivot_income.sort_values(by='ratio', ascending = False))

Доля задолженностей сгруппированных по уровню дохода:
debt_group           задолженность  нет задолженности     ratio
income_category                                                
средний доход                  480               4981  0.087896
высокий доход                  445               4782  0.085135
низкий доход                   427               4918  0.079888
очень высокий доход            381               4963  0.071295


### Вывод

Изучив полученную информацию, можно сделать вывод, что чаще кредит в срок возвращают заёмщики с очень высоким доходом и низким доходом, реже заёмщики со средним и высоким доходами, для них соотвественно вероятность просрочки составляет: 8.78% и 8,51% 

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
print(color.BOLD + 'Доля задолженностей сгруппированных по целям кредита:' + color.END)
data_pivot_purpose =  df.pivot_table(index=['lemmatized_purpose'], columns='debt_group', values='total_income', aggfunc='count')
data_pivot_purpose['ratio'] = data_pivot_purpose['задолженность'] / (data_pivot_purpose['нет задолженности'] + data_pivot_purpose['задолженность'])
print(data_pivot_purpose.sort_values(by='ratio', ascending = False))

Доля задолженностей сгруппированных по целям кредита:
debt_group          задолженность  нет задолженности     ratio
lemmatized_purpose                                            
автомобиль                    401               3889  0.093473
образование                   369               3629  0.092296
свадьба                       183               2132  0.079050
недвижимость                  745               9424  0.073262
ремонт                         35                570  0.057851


### Вывод

Согласно полученным данным, реже всего кредиты возвращают в срок заёмщики, цели заема которых, являются 'автомобиль' и 'образование', вероятность просрочки для них: 9.34% и 9.22% соотвественно. Чаще возвращают займ на ремонт, вероятность просрочки 5.78%.

### №4. Общий вывод

Были получены и обработаны данные более 21000 заёмщиков. Пропуски в данных были обработаны и заполнены статисчески значимой информацией, аномальные значения заменены, удалены дубликаты, лемматизированы цели займа. Также были выделены категории по составу семьи и уровню дохода, эти и другие категории были сгруппированы по наличию задолженностей, благодаря чему удалось подсчитать вероятность просрочки возврата кредита в срок. Таким образом, наибольшие риски имеют:
- Вероятность просрочки для семьей с детьми составляет 9.19%
- Вероятность просрочки для не женатых/не замужних заёмщиков составляет 9.74%, для заёмщиков в гражданском браке 9.30%
- Вероятность просрочки по уровню дохода для среднего составляет 8.78% и 8,51% для высокого
- Вероятность просрочки по целям кредита на покупку автомобиля составляет 9.34% и 9.22% на получение образования

 Пониженный риск к просрочке займа:

- Вероятность просрочки для семьей без детей составляет 7.54%
- Вероятность просрочки для вдов/вдовцов составляет 6.59%, для разведенных заёмщиков в 7.04%
- Вероятность просрочки по уровню дохода для очень высокого составляет 7.12% и 7,98% для низкого
- Вероятность просрочки по целям кредита на ремонт жилья составляет 5.78% и 7.32% на покупку недвижимости
